# Introduction to Modeling Libraries in Python

In this book, I have focused on providing a programming foundation for doing data analysis in Python. Since data analysts and scientists often report spending a disproportionate amount of time with data wrangling and preparation, the book's structure reflects the importance of mastering these techniques.

Which libarary you use for developing models will depend on the application. Many statistical problems can be solved by simpler techniques like orginary least squares regression, while other problems may call for more advanced machine learning methods. Fortunately, Python has become one of the languages of choice for impleting this book.

In this chapter, I will review some features of pandas that may be helpful when you're crossing back and forth between data wrangling with pandas and model fitting and scoring. I will give short introductions to two popular modeling toolkits, stats-models (http://statsmodels.org) and scikit-learn (http://scikit-learn.org). Since each of these projects is large enough to warrant its own dedicated book, I make no effort to be comprehensive and instead direct you to both projects' online documentation along with some other Python-based books on data science, statistics, and machine learning.

## Interfacing Between pandas and Model Code

To turn a DataFrame into a NumPy array, use the __.values__ properties:

In [1]:
import pandas as pd; import numpy as np

In [2]:
data = pd.DataFrame({
    'x0': [1,2,3,4,5],
    'x1': [0.01,-0.01,0.25,-4.1,0.],
    'y': [-1.5, 0., 3.6, 1.3,-2.]
})

In [3]:
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [4]:
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [5]:
data.values

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

To convert back to a DataFrame, you can pass a two-dimensional ndarray with optimal column names:

In [6]:
df2 = pd.DataFrame (data.values, columns=['one','two','three'])

In [7]:
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


In [8]:
df3 = data.copy()

In [9]:
df3['strings']=['a','b','c','d','e']

In [10]:
df3

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [11]:
df3.values

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

In [12]:
model_cols = ['x0','x1']

In [13]:
data.loc[:,model_cols].values

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

In [14]:
data['category']=pd.Categorical(['a','b','a','a','b'],
                               categories=['a','b'])

In [15]:
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


In [16]:
dummies = pd.get_dummies(data.category, prefix='category')

In [17]:
data_with_dummies = data.drop('category',axis=1).join(dummies)

In [18]:
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


## Creating Model Descriptions with Patsy

Patsy (https://patsy.readthedocs.io/) is a Python library for describing statistical models (especially linear models) with a small string-based "formula syntax", which is insprired by (but not exactly the same as) the formula syntax used by the R and S statistical programming languages.

Patsy is well supported for specifying linear models in statsmodels, so I will focus on some of the main features to help you get up and running. Patsy's formulas are special string syntax that looks like:

y ~ x0 + x1

The syntax __a + b__ does not mean to add a to b, but rather that these are terms in the design matrix created for the model. The patsy.dmatices function takes a formula string along with a dataset (which can be a DataFrame or a dict of arrays) and produces design matices for a linear model:

In [19]:
data = pd.DataFrame({
    'x0': [1,2,3,4,5],
    'x1': [0.01,-0.01,0.25,-4.1,0.],
    'y': [-1.5,0.,3.6,1.3,-2.]
})

In [20]:
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [21]:
import patsy

In [22]:
y,X = patsy.dmatrices('y ~ x0 + x1', data)

In [23]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [24]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [25]:
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [26]:
np.array(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

In [28]:
patsy.dmatrices('y ~ x0 + x1 + 0', data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

In [29]:
coef, resid, _, _ = np.linalg.lstsq(X,y)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


In [30]:
coef

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [31]:
coef = pd.Series(coef.squeeze(), index=X.design_info.column_names)

In [32]:
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

### Data Transformations in Patsy Formulas 

In [33]:
y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1)+1)', data)

In [34]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

In [35]:
y, X = patsy.dmatrices('y ~ standardize(x0) + center(x1)', data)

In [36]:
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

In [37]:
new_data = pd.DataFrame({
    'x0': [6,7,8,9],
    'x1': [3.1,-0.5,0,2.3],
    'y': [1,2,3,4]
})

In [38]:
new_X = patsy.build_design_matrices([X.design_info], new_data)

In [39]:
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

In [41]:
y, X = patsy.dmatrices('y ~ I(x0 + x1)', data)

In [42]:
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

Categroical data has a special class of transformations, which will be explained next.

### Categorical Data and Patsy

In [43]:
data = pd.DataFrame({
    'key1': ['a','a','b','b','a','b','a','b'],
    'key2': [0,1,0,1,0,1,0,0],
    'v1': [1,2,3,4,5,6,7,8],
    'v2': [-1,0,2.5,-0.5,4.0,-1.2,0.2,-1.7]
})

In [44]:
y,X = patsy.dmatrices('v2 ~ key1', data)

In [46]:
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

In [47]:
y,X = patsy.dmatrices('v2 ~ key1 + 0', data)

In [48]:
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

Numeric columns can be interpreted as categorical with the C function:

In [49]:
y, X = patsy.dmatrices('v2 ~ C(key2)', data)

In [52]:
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

In [53]:
data['key2'] = data['key2'].map({0: 'zero', 1: 'one'})

In [54]:
data

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [55]:
y, X = patsy.dmatrices('v2 ~ key1 + key2', data)

In [56]:
X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [57]:
y, X = patsy.dmatrices('v2 ~ key1 + key2 + key1:key2', data)

In [58]:
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

## Introduction to statsmodels

### Estimating Linear Models

In [61]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [1]:
x = 4737

In [3]:
c = 73829

In [6]:
r = 3838

In [8]:
y = 2828

In [9]:
h = 2727

### Estimating Time Series Processes

## Introduction to scikit-learn

## Continuing Your Education